## Consegna
Misurazione dell’overlap lessicale tra una serie di definizioni per concetti generici/specifici e concreti/astratti. Partendo dai dati sulle definizioni (presente nella cartella "dati" su Moodle), si richiede di calcolare la similarità 2-a-2 tra le definizioni (ad es. usando la cardinalità dell'intersezione dei lemmi normalizzata sulla lunghezza minima delle definizioni), aggregando (ed effettuando la media degli score di similarità) sulle due dimensioni (concretezza / specificità). 



### Import


In [14]:
import polars as pl
import json

import sys
sys.path.append('..')
from utils import utils as u

### Lettura File


In [15]:
# read csv with polars module
df = json.loads(pl.read_csv("../datasets/TLN-definitions-23.tsv", separator='\t').write_json())

data = {}

### Definizione Struttura dati
La struttura dati utilizzata avrà la seguente forma: 
```json
{
  "name": {
    "1": ["def_1", "def_2", "...", "def_n"],
    "2": ["def_1", "def_2", "...", "def_n"],
    "m": ["def_1", "def_2", "...", "def_n"]
  }
}
```

In [16]:
for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = {}
        count = 1
        for v in col['values']:
            data[col['name']][count] = u.noise_reduction_en(v)
            count += 1


### Calcolo Similarità
Abbiamo calcolato la similarità tra due definizioni utilizzando la cardinalità dell'intersezione tra le definizioni tokenizzate normalizzate per la minima tra le due:
$$
similarity = \frac{|def_1\cap def_2|}{|def_1|} \lor \frac{|def_1\cap def_2|}{|def_2|}
$$

In [17]:
def compute_similarity(definition_1_tokens, definition_2_tokens):

    min_len = 0

    if len(definition_1_tokens) > len(definition_2_tokens):
        min_len = len(definition_2_tokens)
    else:
        min_len = len(definition_1_tokens)

    return (len(set(definition_1_tokens).intersection(set(definition_2_tokens))) / min_len)


### Main

In [18]:
results = {}
final = {}
for word in data:
    results[word] = {}
    for id in data[word]:
        for id_2 in data[word]:
            if id != id_2 and ((id, id_2), (id_2, id)) not in results[word]:
                results[word][((id, id_2), (id_2, id))] = compute_similarity(
                    data[word][id], data[word][id_2])

    final[word] = round((sum(results[word].values()) / \
        len(results[word])if len(results[word]) > 0 else 0), 2)


### Stampa Risultati

In [19]:


data = {
    "": ["Generico", "Specifico"],
    "Astratto": ["pain: "+str(final['pain']), "blurriness: "+str(final['blurriness'])],
    "Concreto": ["door: "+str(final['door']), "ladybug: "+str(final['ladybug'])]
}

table = pl.DataFrame(data)

print(table)


shape: (2, 3)
┌───────────┬──────────────────┬───────────────┐
│           ┆ Astratto         ┆ Concreto      │
│ ---       ┆ ---              ┆ ---           │
│ str       ┆ str              ┆ str           │
╞═══════════╪══════════════════╪═══════════════╡
│ Generico  ┆ pain: 0.17       ┆ door: 0.21    │
│ Specifico ┆ blurriness: 0.07 ┆ ladybug: 0.52 │
└───────────┴──────────────────┴───────────────┘
